# Sampled power analysis

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import itertools
import warnings

import os
import pickle as pkl
%matplotlib inline
import imp

/tmp/ipykernel_3660180/3767633645.py:13: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
import sys
sys.path.append('/home/ubuntu/Github/scrna-parameter-estimation/')
sys.path.append('/home/ubuntu/Github/misc-seq/miscseq/')
# import encode
import memento
# import run_memento

In [5]:
data_path  = '/Data/lupus/'
num_inds = [50, 60, 70, 80]
num_resample=10
cts = ['B', 'T8', 'T4', 'NK', 'cM', 'ncM']

In [6]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", ['turquoise', 'silver', 'slategrey'])


import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'medium',
         'axes.titlesize':'medium',
         'figure.titlesize':'medium',
         'xtick.labelsize':'small',
         'ytick.labelsize':'small'}
pylab.rcParams.update(params)

### Make summation pseudobulks

Matrix eqtl used log(mean+1) pseudobulks

In [16]:
adata = sc.read(data_path + 'Lupus_study_adjusted_counts.h5ad')

/home/ubuntu/anaconda3/envs/mementocxg/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [19]:
pos_asian = pd.read_csv(data_path + 'mateqtl_input/asian_genos.tsv', sep='\t', index_col=0)
pos_eur = pd.read_csv(data_path + 'mateqtl_input/eur_genos.tsv', sep='\t', index_col=0)

In [20]:
cts = ['T4', 'cM', 'ncM', 'T8', 'B', 'NK']
for ct in cts:
        
    subset = adata[(adata.obs['cg_cov'] == ct) & (adata.obs['ind_cov'].isin(pos_asian.columns))].copy()
    # subset.write(data_path + 'single_cell/asian_{}.h5ad'.format(ct))
    asian_ind_list = pos_asian.columns.tolist()

    means = []
    for ind in asian_ind_list:
        view = subset[subset.obs.ind_cov==ind]
        if view.shape[0] == 0:
            means.append(view.X.sum(axis=0).A1)
        else:
            means.append(view.X.sum(axis=0).A1)
    pseudobulk = pd.DataFrame(np.vstack(means), columns=subset.var.index, index=asian_ind_list)
    pseudobulk.to_csv(data_path + 'pseudobulk/asian_{}_sum.csv'.format(ct), sep='\t')
        
    subset = adata[(adata.obs['cg_cov'] == ct) & (adata.obs['ind_cov'].isin(pos_eur.columns))].copy()
    # subset.write(data_path + 'single_cell/eur_{}.h5ad'.format(ct))
    
    eur_ind_list = pos_eur.columns.tolist()

    means = []
    for ind in eur_ind_list:
        view = subset[subset.obs.ind_cov==ind]
        if view.shape[0] == 0:
            means.append(view.X.sum(axis=0).A1)
        else:
            means.append(view.X.sum(axis=0).A1)
    pseudobulk = pd.DataFrame(np.vstack(means), columns=subset.var.index, index=eur_ind_list)
    pseudobulk.to_csv(data_path + 'pseudobulk/eur_{}_sum.csv'.format(ct), sep='\t')
    

### Sample the SNP and covariate dataframes

In [ ]:
def drop_zero_col(df):
    
    return df.loc[:, (df != 0).any(axis=0)].copy()

### Save SNP dataframes

In [42]:
# Read the OneK result
onek_replication = pd.read_csv(data_path + 'filtered_onek_eqtls.csv')

In [43]:
onek_replication.groupby('cg_cov').size()

cg_cov
B      1352
NK     1528
T4     2980
T8     2741
cM      406
ncM     375
dtype: int64

In [44]:
ct_eqtl_snps = onek_replication.query('cg_cov == "T4"')[['rsid']]
ct_eqtl_snps.to_csv('one1k_T4_eqtl_snps.csv', index=False)